論文懶人包：<br>
https://github.com/aronwc/mlsm/blob/master/papers/bollen10twitter/vlandeir.md<br>
https://zhuanlan.zhihu.com/p/19944548<br>
使用到：<br>
https://github.com/kbathina/sentiment-tools

They create their own tool : Google-Profile Of Mood States (GPOMS), an extended version of POMS-bi. It is based on a lexicon of 964 terms and can measure human mood states in six mood dimensions : Calm, Alert, Sure, Vital, Kind and Happy.<br>

1. Calm: composed/anxious
2. Kind: agreeable/hostile
3. Happy: elated/depressed
4. Alert: confident/unsure
5. Sure: clearheaded/confuse
6. Vital: denergetic/tired

[詞彙表總覽]
https://github.com/kbathina/sentiment-tools/blob/master/moodscores/data/GPOMS.csv

In [1]:
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import glob
import collections
import re

# I. load tweets Data

In [2]:
condense_df = pd.read_pickle("corpus.pkl")
condense_df.head()

,tweets_processed,tweet_size
date_processed,,
2022-05-01,"[customized, operator, training, helping, team...",142
2022-05-02,"[power, higher, education, one, network, time,...",413
2022-05-03,"[ambulances, make, india, bharti, airtel, hosp...",425
2022-05-04,"[cisco, calculates, emissions, products, math,...",438
2022-05-05,"[employees, said, able, work, remotely, made, ...",474


# II. Process the word list of sentiment

    try! 

In [4]:
tweet = 'This tool is easy and fun to use!!!'

    pre emotion: 996 words!

In [5]:
from moodscores import gpoms
GPOMS = gpoms.GPOMS()

sentiment_gpoms_sum, tokens_gpoms_sum = GPOMS.Score(tweet,'Sum')
sentiment_gpoms_ave, tokens_gpoms_ave = GPOMS.Score(tweet,'Average')

# anxious, hostile, depressed, unsure, confused, tired
for dimension in sentiment_gpoms_sum.keys():
    print(dimension + ' score (Sum) =', sentiment_gpoms_sum[dimension])
    print(dimension + ' score (Average) =', sentiment_gpoms_ave[dimension])

print('Tokenized list =', tokens_gpoms_sum)

composed/anxious score (Sum) = 0.16
composed/anxious score (Average) = 0.08
agreeable/hostile score (Sum) = 0.436
agreeable/hostile score (Average) = 0.218
elated/depressed score (Sum) = 0.237
elated/depressed score (Average) = 0.237
confident/unsure score (Sum) = 0.763
confident/unsure score (Average) = 0.3815
clearheaded/confused score (Sum) = 0.14300000000000002
clearheaded/confused score (Average) = 0.07150000000000001
energetic/tired score (Sum) = 0.2509999999999999
energetic/tired score (Average) = 0.12549999999999994
Tokenized list = {'composed/anxious': ['easy', 'fun'], 'agreeable/hostile': ['easy', 'fun'], 'elated/depressed': ['fun'], 'confident/unsure': ['easy', 'fun'], 'clearheaded/confused': ['easy', 'fun'], 'energetic/tired': ['easy', 'fun']}


# III. Count the ratio of positive v.s. negative on the same day

In [5]:
merge_df = condense_df.copy()

def pre(list):
    return " ".join(list)
merge_df['GPOMS_preprocess_data'] = merge_df['tweets_processed'].map(pre)

def sentiment_GPOMS_sum(tweet):
    sentiment_gpoms_sum, tokens_gpoms_sum = GPOMS.Score(tweet,'Sum')
    tmp = []
    for dimension in sentiment_gpoms_sum.keys():
        tmp.append(sentiment_gpoms_sum[dimension])

    return tmp

def sentiment_GPOMS_avg(tweet):
    sentiment_gpoms_ave, tokens_gpoms_ave = GPOMS.Score(tweet,'Average')
    tmp = []
    for dimension in sentiment_gpoms_sum.keys():
        tmp.append(sentiment_gpoms_ave[dimension])

    return tmp


merge_df['GPOMS_sum'] = merge_df['GPOMS_preprocess_data'].map(sentiment_GPOMS_sum)
merge_df['GPOMS_avg'] = merge_df['GPOMS_preprocess_data'].map(sentiment_GPOMS_avg)

In [6]:
merge_df

,tweets_processed,tweet_size,GPOMS_preprocess_data,GPOMS_sum,GPOMS_avg
date_processed,,,,,
2022-05-01,"[customized, operator, training, helping, team...",142,customized operator training helping team morg...,"[0.455, 0.5309999999999998, -0.008, 2.529, 0.5...","[0.11375, 0.13274999999999995, -0.002, 0.4215,..."
2022-05-02,"[power, higher, education, one, network, time,...",413,power higher education one network time check ...,"[0.29500000000000004, 2.207, -1.18399999999999...","[0.036875000000000005, 0.16976923076923076, -0..."
2022-05-03,"[ambulances, make, india, bharti, airtel, hosp...",425,ambulances make india bharti airtel hospital p...,"[0.102, 0.7470000000000002, -1.802000000000000...","[0.020399999999999998, 0.09337500000000003, -0..."
2022-05-04,"[cisco, calculates, emissions, products, math,...",438,cisco calculates emissions products math whiz ...,"[2.695, 2.8609999999999998, -2.491, 10.7340000...","[0.19249999999999998, 0.16829411764705882, -0...."
2022-05-05,"[employees, said, able, work, remotely, made, ...",474,employees said able work remotely made learn t...,"[-1.8730000000000002, 1.2449999999999999, -1.4...","[-0.18730000000000002, 0.1383333333333333, -0...."
...,...,...,...,...,...
2022-06-26,"[follow, final, day, action, sundays, trophies...",54,follow final day action sundays trophies trave...,"[0, 0, 0, 1.0, 0, 0]","[0, 0, 0, 1.0, 0, 0]"
2022-06-27,"[hola, enviarnos, un, mensaje, privado, desde,...",459,hola enviarnos un mensaje privado desde donde ...,"[1.3619999999999999, 1.3589999999999995, -0.44...","[0.22699999999999998, 0.11324999999999996, -0...."
2022-06-28,"[ciscolive, keynote, roundup, las, ante, learn...",444,ciscolive keynote roundup las ante learn url b...,"[0.10099999999999973, 1.7439999999999998, -1.4...","[0.010099999999999974, 0.15854545454545452, -0..."


In [7]:
def max_index(list):
    return list.index(max(list))

In [8]:
merge_df['sum_index'] = merge_df['GPOMS_sum'].map(max_index)
merge_df['avg_index'] = merge_df['GPOMS_avg'].map(max_index)

In [9]:
merge_df.head()

,tweets_processed,tweet_size,GPOMS_preprocess_data,GPOMS_sum,GPOMS_avg,sum_index,avg_index
date_processed,,,,,,,
2022-05-01,"[customized, operator, training, helping, team...",142,customized operator training helping team morg...,"[0.455, 0.5309999999999998, -0.008, 2.529, 0.5...","[0.11375, 0.13274999999999995, -0.002, 0.4215,...",3,3
2022-05-02,"[power, higher, education, one, network, time,...",413,power higher education one network time check ...,"[0.29500000000000004, 2.207, -1.18399999999999...","[0.036875000000000005, 0.16976923076923076, -0...",3,3
2022-05-03,"[ambulances, make, india, bharti, airtel, hosp...",425,ambulances make india bharti airtel hospital p...,"[0.102, 0.7470000000000002, -1.802000000000000...","[0.020399999999999998, 0.09337500000000003, -0...",3,3
2022-05-04,"[cisco, calculates, emissions, products, math,...",438,cisco calculates emissions products math whiz ...,"[2.695, 2.8609999999999998, -2.491, 10.7340000...","[0.19249999999999998, 0.16829411764705882, -0....",3,3
2022-05-05,"[employees, said, able, work, remotely, made, ...",474,employees said able work remotely made learn t...,"[-1.8730000000000002, 1.2449999999999999, -1.4...","[-0.18730000000000002, 0.1383333333333333, -0....",3,3


In [12]:
# merge_df.to_pickle("GPOMS_res.pkl")

In [10]:
result = pd.DataFrame(merge_df, columns = ["tweet_size", "avg_index"])
result 

,tweet_size,avg_index
date_processed,,
2022-05-01,142,3
2022-05-02,413,3
2022-05-03,425,3
2022-05-04,438,3
2022-05-05,474,3
...,...,...
2022-06-26,54,3
2022-06-27,459,3
2022-06-28,444,3


In [12]:
result.describe()

,tweet_size,avg_index
count,61.000000,61.000000
mean,437.245902,2.934426
std,258.296061,0.813815
min,54.000000,0.000000
25%,158.000000,3.000000
50%,459.000000,3.000000
75%,602.000000,3.000000
max,1119.000000,5.000000


### 加入絕對值再去做比較

In [30]:
import pickle5 as pickle

In [31]:
with open("GPOMS_res.pkl","rb") as f:
    GPOMS = pickle.load(f)
    
GPOMS = GPOMS.drop(["tweets_processed", "tweet_size", "GPOMS_preprocess_data", "sum_index", "avg_index"], axis=1)
GPOMS.head()

,GPOMS_sum,GPOMS_avg
date_processed,,
2022-05-01,"[0.455, 0.5309999999999998, -0.008, 2.529, 0.5...","[0.11375, 0.13274999999999995, -0.002, 0.4215,..."
2022-05-02,"[0.29500000000000004, 2.207, -1.18399999999999...","[0.036875000000000005, 0.16976923076923076, -0..."
2022-05-03,"[0.102, 0.7470000000000002, -1.802000000000000...","[0.020399999999999998, 0.09337500000000003, -0..."
2022-05-04,"[2.695, 2.8609999999999998, -2.491, 10.7340000...","[0.19249999999999998, 0.16829411764705882, -0...."
2022-05-05,"[-1.8730000000000002, 1.2449999999999999, -1.4...","[-0.18730000000000002, 0.1383333333333333, -0...."


In [35]:
GPOMS_prim = GPOMS.copy()
GPOMS_prim[['Calm','Kind','Happy','Alert','Sure','Vital']] = pd.DataFrame(GPOMS.GPOMS_avg.tolist(), index= GPOMS.index)
GPOMS_prim = GPOMS_prim.drop(['GPOMS_sum', 'GPOMS_avg'], axis=1)
GPOMS_prim

,Calm,Kind,Happy,Alert,Sure,Vital
date_processed,,,,,,
2022-05-01,0.113750,0.132750,-0.002000,0.421500,0.104200,-0.211500
2022-05-02,0.036875,0.169769,-0.098667,0.329500,0.102600,-0.021833
2022-05-03,0.020400,0.093375,-0.163818,0.557714,0.141286,0.215077
2022-05-04,0.192500,0.168294,-0.166067,0.447250,0.101750,0.180778
2022-05-05,-0.187300,0.138333,-0.159667,0.312538,0.173375,0.140182
...,...,...,...,...,...,...
2022-06-26,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2022-06-27,0.227000,0.113250,-0.049333,0.580214,0.192500,0.191100
2022-06-28,0.010100,0.158545,-0.128545,0.377867,0.196909,0.092214


In [25]:
# GPOMS_prim['Calm_pvn'] = (GPOMS_prim['Calm']>0)
# GPOMS_prim['Kind_pvn'] = (GPOMS_prim['Kind']>0)
# GPOMS_prim['Happy_pvn'] = (GPOMS_prim['Happy']>0)
# GPOMS_prim['Alert_pvn'] = (GPOMS_prim['Alert']>0)
# GPOMS_prim['Sure_pvn'] = (GPOMS_prim['Sure']>0)
# GPOMS_prim['Vital_pvn'] = (GPOMS_prim['Vital']>0)

In [38]:
GPOMS_prim['Max_value'] = GPOMS_prim.apply(lambda x: max(x.min(), x.max(), key=abs),axis=1)
GPOMS_prim['Max_type'] = GPOMS_prim.idxmax(axis=1)
GPOMS_prim.head()

,Calm,Kind,Happy,Alert,Sure,Vital,Max_value,Max_type
date_processed,,,,,,,,
2022-05-01,0.113750,0.132750,-0.002000,0.421500,0.104200,-0.211500,0.421500,Alert
2022-05-02,0.036875,0.169769,-0.098667,0.329500,0.102600,-0.021833,0.329500,Alert
2022-05-03,0.020400,0.093375,-0.163818,0.557714,0.141286,0.215077,0.557714,Alert
2022-05-04,0.192500,0.168294,-0.166067,0.447250,0.101750,0.180778,0.447250,Alert
2022-05-05,-0.187300,0.138333,-0.159667,0.312538,0.173375,0.140182,0.312538,Alert


In [39]:
GPOMS_prim['Max_type'].describe()

count        61
unique        4
top       Alert
freq         54
Name: Max_type, dtype: object

In [18]:
for d in GPOMS_prim:
    print(d)
#     tmp = d.index(max(d))
#     print(tmp)

Calm
Kind
Happy
Alert
Sure
Vital
